In [ ]:
%pylab inline
plt.ion()

#### Import the task we want and set a couple of defaults

In [ ]:
from lsst.eo_utils.bias.bias_v_row import BiasVRowTask

In [ ]:
task = BiasVRowTask(bias='spline', plot='display')

#### Check the configuration of the task

In [ ]:
task.config

#### Make an iterator for this task and check the configuration of that iterator

In [ ]:
itr = task.make_iterator()

In [ ]:
itr.config

#### Use the iterator to run the task on one slot for one run

In [ ]:
itr.run_with_args(butler_repo='TS8', runs=['6106D'], slots=['S00'], nfiles=2)

#### The line of code above is equivalent to the following

In [ ]:
from lsst.eo_utils.base.butler_utils import get_butler_by_repo
butler = get_butler_by_repo('TS8')

In [ ]:
data = itr.get_data(butler, '6106D', nfiles=2)

In [ ]:
raft_data = data['RTM-004']

In [ ]:
slot_data = raft_data['S00']

In [ ]:
task(butler, slot_data)